In [59]:
! rm -rf ""

In [29]:
!pip install obonet
!pip install pyvis

In [30]:
import re
def get_embeddings(seq):
    sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq)))]

    ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")

    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # generate embeddings
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,
                               attention_mask=attention_mask)

    # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
    emb_0 = embedding_repr.last_hidden_state[0]
    emb_0_per_protein = emb_0.mean(dim=0)

    return emb_0_per_protein

In [31]:
import os
import json
from typing import Dict
from collections import Counter

import random
import obonet
import pandas as pd
import numpy as np
from Bio import SeqIO

from transformers import T5Tokenizer, T5EncoderModel
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False) #.to(device)

# Load the model
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device);

# only GPUs support half-precision currently; if you want to run on CPU use full-precision (not recommended, much slower)
#model.full() if device=='cpu' else model.half()

In [32]:
! pip install Bio

In [34]:
predictions= [[1 for i in range(1024)]]
a=pd.DataFrame(predictions)
a.to_csv("/content/input.csv")

In [80]:
import tqdm
from Bio import SeqIO
import numpy as np
import pandas as pd
import tensorflow as tf

def predict(filepath):

    fn = filepath

    sequences = SeqIO.parse(fn, "fasta")

    ids = []
    num_sequences=sum(1 for seq in sequences)
    embeds = np.zeros((num_sequences, 1024))
    i = 0
    for seq in tqdm.tqdm(sequences):
        ids.append(seq.id)
        embeds[i] = get_embeddings(str(seq.seq)).detach().cpu().numpy()
        i += 1
        # if i==10:
        #     break #remove it for full calculation

    # np.save('train_embeds.npy', embeds)
    # np.save('train_ids.npy', np.array(ids))
    # batch_shape=5120
    INPUT_SHAPE=[1024]
    num_of_labels=1500
    # from tf.keras.models import load_model
    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['binary_accuracy', tf.keras.metrics.AUC()]
    )
    model.load_weights('/content/drive/MyDrive/my_model.weights.h5') #load model here
    labels_df=pd.read_csv('/content/truncated.csv')
    labels_df=labels_df.drop(columns='Unnamed: 0')
#     custom_input_tensor = np.load("/kaggle/input/t5embeds/test_embeds.npy") # add the numpy array here
    custom_input_tensor=embeds
    # custom_input_tensor=np.zeros((3, 1024))
    print(custom_input_tensor)
    print(len(custom_input_tensor[0]))
    # Get predictions for custom input tensor
    predictions = model.predict(custom_input_tensor)
    predictions_list=[]
    # 'predictions' will contain the model's output for the custom input tensor
    # print(predictions)
    for prediction in predictions:
        tmp=[]
        for i in prediction:
            x=0 if i<0.5 else 1
            tmp.append(x)
        predictions_list.append(tmp.copy())
        # print(sum(tmp))
        print(prediction)
    # print(len(predictions))

    # labels_df=
    # Get the column names (labels) from the original DataFrame
    label_columns = labels_df.columns

    # Convert the predictions into a DataFrame
    predictions_df = pd.DataFrame(predictions_list, columns=label_columns)

    # Save the DataFrame to a CSV file
    predictions_df.to_csv("custom_predictions.csv", index=False) #output csv
    return "custom_predictions.csv"


In [4]:
!pip install -q gradio

In [77]:
a=predict("/content/abc.fasta")
print(a)

0it [00:00, ?it/s]


[[0. 0. 0. ... 0. 0. 0.]]
1024
1/1 [==============================] - 0s 202ms/step
1
custom_predictions.csv


In [75]:
# batch_shape=5120
import tensorflow as tf
import numpy as np
import pandas as pd

INPUT_SHAPE=[1024]
num_of_labels=1500
# from tf.keras.models import load_model
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', tf.keras.metrics.AUC()]
)
model.load_weights('/content/drive/MyDrive/my_model.weights.h5')

# model=tf.keras.models.load_model("/content/drive/MyDrive/model1.h5") #load model here
labels_df=pd.read_csv('/content/truncated.csv')
labels_df=labels_df.drop(columns='Unnamed: 0')
#     custom_input_tensor = np.load("/kaggle/input/t5embeds/test_embeds.npy") # add the numpy array here
# custom_input_tensor=embeds
custom_input_tensor=np.random.random((100, 1024))
print(custom_input_tensor)
print(len(custom_input_tensor[0]))
# Get predictions for custom input tensor
predictions = model.predict(custom_input_tensor)
predictions_list=[]
# 'predictions' will contain the model's output for the custom input tensor
# print(predictions)
cnt=0
for prediction in predictions:
    tmp=[]
    for i in prediction:
        x=0 if i<0.5 else 1
        tmp.append(x)
    predictions_list.append(tmp.copy())
    # print(sum(tmp))
    cnt+=sum(tmp)
    # print(tmp)
    print(sum(tmp))
print(cnt)
# print(len(predictions))

# labels_df=
# Get the column names (labels) from the original DataFrame
label_columns = labels_df.columns

# Convert the predictions into a DataFrame
predictions_df = pd.DataFrame(predictions_list, columns=label_columns)

# Save the DataFrame to a CSV file
predictions_df.to_csv("custom_predictions.csv", index=False) #output csv

[[0.04404056 0.08912223 0.26120968 ... 0.62678027 0.82235433 0.3202845 ]
 [0.24504959 0.87050151 0.282636   ... 0.26825446 0.96879317 0.17777832]
 [0.85421075 0.66590741 0.5749779  ... 0.89506666 0.54754993 0.03731561]
 ...
 [0.13289982 0.87559716 0.3199096  ... 0.1790593  0.97360167 0.22326068]
 [0.40723982 0.82847947 0.46348826 ... 0.52270772 0.90143254 0.35830025]
 [0.04749935 0.42091512 0.91150753 ... 0.15370932 0.68776259 0.81459474]]
1024
4/4 [==============================] - 0s 10ms/step
7
4
4
5
4
3
2
7
3
4
3
5
4
4
3
7
2
4
3
9
8
6
5
4
7
3
4
3
5
4
1
4
5
4
2
3
4
8
4
3
3
8
7
5
4
5
3
3
4
5
2
8
4
5
5
2
3
2
6
5
7
4
4
5
5
4
5
3
3
5
2
4
3
5
5
4
2
3
1
12
4
7
9
2
5
7
6
4
6
4
2
4
5
5
3
2
6
3
4
4
441


In [16]:
import gradio as gr
import pandas as pd


In [81]:
import gradio as gr
gr.Interface(
    predict,
    title = 'Protein Function Prediction using fasta file,upload a fasta file',
    inputs="file",
    outputs="file"
).launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f0948d36ed720d5ae5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0it [00:00, ?it/s]


[[0. 0. 0. ... 0. 0. 0.]]
1024
1/1 [==============================] - 0s 142ms/step
[0.4348464  0.53423107 0.42620808 ... 0.08446681 0.08856744 0.14838184]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d8b932897f950af0db.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://d0b2de10a3ab82e50e.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://5541d58b12b1a61bc0.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://b3dbaf6fd558c49a43.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://5c3a55f6a215182ca7.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://af815fe6c84532cecb.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://f0948d36ed720d5ae5.gradio.live
